# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
data=pd.read_csv("../weatherPy/weather_data.csv")
data.drop('Unnamed: 0',axis=1,inplace=True)
data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,rikitea,-23.12,-134.97,70.07,80.0,18.0,22.62,PF,1.595714e+09
1,dzilam gonzalez,21.28,-88.93,89.49,59.0,100.0,10.20,MX,1.595714e+09
2,albany,42.60,-73.97,90.00,36.0,100.0,3.94,US,1.595713e+09
3,oistins,13.07,-59.53,86.00,74.0,40.0,13.87,BB,1.595714e+09
4,ostrovnoy,68.05,39.51,57.87,80.0,59.0,6.17,RU,1.595714e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)
#configure gamps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
center=(40.416775,-3.703790) #spain as the center
zoom_level=5
fig = gmaps.figure(layout=figure_layout,zoom_level=zoom_level,center=center)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [17]:
locations=[(lat,lng) for lat,lng in zip(data['Lat'],data['Lng'])]
humidity=data['Humidity']

In [18]:
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

Narrow down the DataFrame to find your own ideal weather conditions. For example:
Maximum temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number like 10.






In [19]:
conditions=(data['Max Temp']<75)&(data['Max Temp']>65)&(data['Wind Speed']<8)&(data['Cloudiness']<60)
hh_data=data[conditions]
print(f" {len(hh_data)} ideal options")
hh_data.head()

 35 ideal options


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,dindori,22.95,81.08,72.64,76.0,3.0,2.46,IN,1.595714e+09
47,bima,-8.47,118.72,69.04,88.0,56.0,3.60,ID,1.595714e+09
48,cochrane,51.18,-114.47,69.01,48.0,1.0,3.36,CA,1.595714e+09
52,vityazevo,44.99,37.27,73.40,53.0,40.0,4.47,RU,1.595714e+09
55,bengkulu,-3.80,102.27,74.12,90.0,17.0,4.70,ID,1.595714e+09


In [20]:
#configure gamps
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,center=center,zoom_level=zoom_level)
fig

locations=[(lat,lng) for lat,lng in zip(hh_data['Lat'],hh_data['Lng'])]
humidity=hh_data['Humidity']
heat_layer = gmaps.heatmap_layer(locations=locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotel_data={}


for location,city,country in zip(locations,hh_data['City'],hh_data['Country']):
    params = {
    "location": f'{location[0]}, {location[1]}',
    "rankby": "distance",
    "type": "hotel",
    "key": g_key}
    response = requests.get(base_url, params=params)
    data=response.json()
    for i in range(len(data['results'])):            
        name=data['results'][i]['name']
        lat=data['results'][i]["geometry"]['location']['lat']
        lng=data['results'][i]["geometry"]['location']['lng']
        hotel_data.update({name:[name,country,city,lat,lng]})

    

In [22]:
print(f'We have {len(hotel_data)} hotel options')

We have 576 hotel options


In [23]:
hotel_df=pd.DataFrame.from_dict(hotel_data, orient='index',columns=['Hotel Name','Country','City','Lat','Lng'])
hotel_df.head()

,Hotel Name,Country,City,Lat,Lng
Shorya online and printing press,Shorya online and printing press,IN,dindori,22.949711,81.080184
N.P. ASSOCIATES,N.P. ASSOCIATES,IN,dindori,22.949724,81.080215
Narmada Traders,Narmada Traders,IN,dindori,22.949361,81.079973
Narmada Traders Pro. Suneel Dutt Tiwari,Narmada Traders Pro. Suneel Dutt Tiwari,IN,dindori,22.949265,81.079997
Chhavi Lal Nagesh House,Chhavi Lal Nagesh House,IN,dindori,22.948939,81.079823


In [24]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [28]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…